# Trabalho Final - Introdução à Teoria da Informação

##### Ana Luisa Miranda - 20200017005
##### Helio Lima Correia - 202000123868

In [10]:
import random
import os

In [11]:
def adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia):
    proximo_codigo = len(dicionario)
    dicionario[nova_sequencia] = proximo_codigo

In [12]:
def criar_dicionario_inicial():
    dicionario = {}
    for i in range(256):
        dicionario[(i.to_bytes(1,byteorder='big',signed=False))] = i
    return dicionario

In [13]:
def compressao_LZW(nome_arquivo, tamanho_maximo):
    dicionario = criar_dicionario_inicial()
    tam_max_dic = 2 ** tamanho_maximo
    buffer = b''
    codigos = []

    with open(nome_arquivo, 'rb') as arquivo:
        while True:
            byte = arquivo.read(1)

            if not byte:
                break
            
            nova_sequencia = buffer + byte

            if nova_sequencia in dicionario:
                buffer = nova_sequencia
            else:
                if(len(dicionario) < tam_max_dic): # verifica tamanho maximo do dicionario
                    adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia)
                codigos.append(dicionario[buffer])
                buffer = byte

    if buffer:
        codigos.append(dicionario[buffer])
    #print(dicionario)
    return dicionario

In [14]:
def rec_pad(nome_arquivo, dicionario):
    #tam_max_dic = 2 ** tamanho_maximo
    buffer = b''
    codigos = []

    with open(nome_arquivo, 'rb') as arquivo:
        cabe = arquivo.read(14)
        #print(cabe)
        while True:
            byte = arquivo.read(1)

            if not byte:
                break
            
            nova_sequencia = buffer + byte

            if nova_sequencia in dicionario:
                buffer = nova_sequencia
            else:
                codigos.append(dicionario[buffer])
                buffer = byte

    if buffer:
        codigos.append(dicionario[buffer])
    
    return len(codigos)

In [15]:
def concatena(arquivos):

    # Nome do arquivo de saída
    arquivo_saida = 'saida.bin'

    # Abrir o arquivo de saída em modo de escrita
    with open(arquivo_saida, 'wb') as arquivo_final:
        # Iterar sobre a lista de arquivos
        for nome_arquivo in arquivos:
            # Abrir cada arquivo de entrada em modo de leitura
            with open(nome_arquivo, 'rb') as arquivo:
                #retira o cabeçalho
                cabe = arquivo.read(14)
                #print(cabe)
                # Ler o conteúdo do arquivo de entrada
                conteudo = arquivo.read()
                #print(conteudo)
                # Escrever o conteúdo no arquivo de saída
                arquivo_final.write(conteudo)
                
            # Adicionar uma quebra de linha entre os arquivos (opcional)
            #arquivo_final.write('\n')
    return arquivo_saida
 

In [16]:
bancos = 40
faces = []
amostras = []

for i in range(bancos):
    pessoa = [f'orl_faces/orl_faces/s{i+1}/1.pgm', f'orl_faces/orl_faces/s{i+1}/2.pgm', f'orl_faces/orl_faces/s{i+1}/3.pgm', f'orl_faces/orl_faces/s{i+1}/4.pgm', f'orl_faces/orl_faces/s{i+1}/5.pgm', f'orl_faces/orl_faces/s{i+1}/6.pgm', f'orl_faces/orl_faces/s{i+1}/7.pgm', f'orl_faces/orl_faces/s{i+1}/8.pgm', f'orl_faces/orl_faces/s{i+1}/9.pgm', f'orl_faces/orl_faces/s{i+1}/10.pgm']
    faces.append(pessoa)

for i in faces:
    indice_aleatorio = random.randint(0, 9)
    amostras.append(i.pop(indice_aleatorio))

print(amostras)
print(f'numero de amostras = {len(amostras)}')

['orl_faces/orl_faces/s1/4.pgm', 'orl_faces/orl_faces/s2/4.pgm', 'orl_faces/orl_faces/s3/5.pgm', 'orl_faces/orl_faces/s4/6.pgm', 'orl_faces/orl_faces/s5/1.pgm', 'orl_faces/orl_faces/s6/9.pgm', 'orl_faces/orl_faces/s7/6.pgm', 'orl_faces/orl_faces/s8/1.pgm', 'orl_faces/orl_faces/s9/1.pgm', 'orl_faces/orl_faces/s10/1.pgm', 'orl_faces/orl_faces/s11/2.pgm', 'orl_faces/orl_faces/s12/4.pgm', 'orl_faces/orl_faces/s13/6.pgm', 'orl_faces/orl_faces/s14/7.pgm', 'orl_faces/orl_faces/s15/6.pgm', 'orl_faces/orl_faces/s16/3.pgm', 'orl_faces/orl_faces/s17/10.pgm', 'orl_faces/orl_faces/s18/1.pgm', 'orl_faces/orl_faces/s19/2.pgm', 'orl_faces/orl_faces/s20/8.pgm', 'orl_faces/orl_faces/s21/6.pgm', 'orl_faces/orl_faces/s22/3.pgm', 'orl_faces/orl_faces/s23/10.pgm', 'orl_faces/orl_faces/s24/4.pgm', 'orl_faces/orl_faces/s25/4.pgm', 'orl_faces/orl_faces/s26/2.pgm', 'orl_faces/orl_faces/s27/9.pgm', 'orl_faces/orl_faces/s28/3.pgm', 'orl_faces/orl_faces/s29/1.pgm', 'orl_faces/orl_faces/s30/8.pgm', 'orl_faces/orl_f

In [17]:
RCs = []
aux2 = []
aux3 = []

for k in range(9,17):

    dicionarios = []

    for i in range(bancos):
        aux = concatena(faces[i])
        dicionarios.append(compressao_LZW(aux,k))

    #print(len(dicionarios))

    for i in amostras:
        for j in dicionarios:
            compressao = rec_pad(i, j)
            file_stats = os.stat(i)
            RC = compressao/file_stats.st_size
            aux2.append(RC)
        #print(len(aux2),aux2)
        aux3.append(aux2.copy())
        aux2.clear()
    RCs.append(aux3.copy())
    aux3.clear()
#print(RCs)
#print(len(RCs))

In [18]:
for k in range(8):
    erro = 0
    acertos = 0
    print(f'k = {k+9}')
    for i in range(len(amostras)):
        #print(f'amostra {i+1}')
        menor = RCs[k][i][i]
        #print(f'prime = {menor}')
        for j in range(len(dicionarios)):
            if(RCs[k][i][j] < menor):
                menor = RCs[k][i][j]
            #print(f'dicionario {j+1}: {RCs[k][i][j]}')
        if(RCs[k][i][i] == menor):
            acertos += 1
        else:
            erro += 1
        #print(f'menor = {menor}')
       
    print(f'acertos = {acertos/(len(amostras))*100}%, erros = {erro/(len(amostras))*100}%')

k = 9
acertos = 25.0%, erros = 75.0%
k = 10
acertos = 20.0%, erros = 80.0%
k = 11
acertos = 15.0%, erros = 85.0%
k = 12
acertos = 32.5%, erros = 67.5%
k = 13
acertos = 67.5%, erros = 32.5%
k = 14
acertos = 60.0%, erros = 40.0%
k = 15
acertos = 92.5%, erros = 7.5%
k = 16
acertos = 95.0%, erros = 5.0%
